In [1]:
import logging
import sys

sys.path.append("../")  # go to parent dir

import os
from dotenv import load_dotenv
import json
from OptiHPLCHandler import EmpowerHandler

# get api address from .env file
load_dotenv("../../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_TST")
EMPOWER_PROJECT_LIST = os.getenv("EMPOWER_PROJECT_LIST")
EMPOWER_PROJECT_LIST = json.loads(EMPOWER_PROJECT_LIST)
connection_file = "connections.json"


logging_handler = logging.FileHandler("new_api.log")
logging_handler.setFormatter(
    logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
)
opti_hplc_handler_logger = logging.getLogger("OptiHPLCHandler")
opti_hplc_handler_logger.setLevel(logging.DEBUG)
opti_hplc_handler_logger.addHandler(logging_handler)

In [2]:
# The unhandled exception handler, which will log the exception as critical
# and then call the default excepthook.
def handle_unhandled_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        # Will call default excepthook
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return
        # Create a critical level log message with info from the except hook.
    opti_hplc_handler_logger.critical(
        "Unhandled exception", exc_info=(exc_type, exc_value, exc_traceback)
    )


# Assign the excepthook to the handler
sys.excepthook = handle_unhandled_exception

In [9]:
connection_list = []
for i in range(1):
    project = EMPOWER_PROJECT_LIST[i]
    if i < 6:
        api_version = "2.0"
    else:
        api_version = "1.0"
    handler = EmpowerHandler(
        project=project,
        address=EMPOWER_API_ADDRESS,
        allow_login_without_context_manager=True,
        username="srfu",
    )
    handler.connection.api_version = api_version
    print(f"Logging in to {project} with api version {api_version}")
    handler.login()
    connection_list.append(
        {
            "api_address": EMPOWER_API_ADDRESS,
            "token": handler.connection.token,
            "project": project,
            "session_id": handler.connection.session_id,
            "login_api_version": api_version,
            "service": handler.connection.service,
        }
    )
    # switch api version, so we test both versions on connections that has been logged
    # in using both versions
    # if 1%2 == 0:
    #    handler.connection.api_version = "1.0"
    # else:
    #    handler.connection.api_version = "2.0"
    method_list = handler.GetMethodList()
    if len(method_list) > 0:
        single_method = handler.GetMethodSetMethod(method_list[0])
    # Remove the session id and token, so it can't logout when I delete the handler by
    # overwriting the "handler" variable
    handler.connection.session_id = None
    handler.connection.token = None
with open(connection_file, "w") as f:
    json.dump(connection_list, f, indent=4)

Logging in to SLCB_2024_EBOFU with api version 2.0


C:\Users\srfu\programming\OptiHPLCHandler\src\OptiHPLCHandler\empower_handler.py:151: UserWarning: You are logging in manually without a context manager. This is not recommended.
Please use a context manager, e.g.
`with EmpowerHandler(...) as handler:...`
  warnings.warn(


In [4]:
import time

time.sleep(60 * 35)
# The tokens on the test server are valid for 30 minutes, so we wait for 35 minutes to
# be sure that the tokens are expired

with open(connection_file, "r") as f:
    connection_list = json.load(f)
for i, connection in enumerate(connection_list):
    handler = EmpowerHandler(
        project=connection["project"],
        address=connection["api_address"],
        service=connection["service"],
    )
    handler.connection.api_version = connection["login_api_version"]
    handler.connection.token = connection["token"]
    handler.connection.session_id = connection["session_id"]
    print(
        f"testing connection to {connection['project']} with api version {handler.connection.api_version}"
    )
    handler.GetMethodList()
    handler.GetNodeNames()
    assert handler.connection.token != connection["token"]

testing connection to SLCB_2024_EBOFU with api version 2.0
